# Monintoring

In [1]:
import requests
import prefect
import pandas as pd
import datetime

from joblib import dump, load
from tqdm.auto import tqdm

In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [26]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric
from evidently.metrics import ColumnQuantileMetric

In [20]:
url="https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet"
resp = requests.get(url, stream = True)

with open('./data/green_tripdata_2024-03.parquet', "wb") as f_in:
    for data in tqdm(resp.iter_content(chunk_size = 1024),
                        unit = "KB"
                        ):
        f_in.write(data)
        
        


0KB [00:00, ?KB/s]

In [33]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


In [34]:
mar_data = read_dataframe('./data/green_tripdata_2024-03.parquet')

In [37]:
mar_data.shape

(55139, 21)

In [39]:
mar_data.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'duration'],
      dtype='object')

In [ ]:
# data labelling

target = ['duration']
num_features = ['fare_amount', 'trip_distance']
cat_features = ['PULocationID', 'DOLocationID']

In [44]:
mar_data[num_features+cat_features]

,fare_amount,trip_distance,PULocationID,DOLocationID
0,12.80,1.72,129,226
1,17.70,3.25,130,218
2,23.30,4.58,255,107
3,22.50,0.00,181,71
4,8.60,1.15,95,135
...,...,...,...,...
57452,12.08,1.45,25,61
57453,12.24,1.13,41,42
57454,12.08,13062.08,223,7
57455,40.52,7.96,42,249


In [46]:
# split test
X_train, X_test, y_train, y_test = train_test_split(mar_data[num_features+cat_features], mar_data[target], test_size=0.2, random_state=42)


In [47]:
model = LinearRegression()

In [49]:
model.fit(X_train, y_train)

LinearRegression()